## Data Preprocessing
Luca Santarella - University of Pisa - 21/07/2023

In [ ]:
import os
import time
import numpy as np
import pandas as pd
import datetime
from pycoingecko import CoinGeckoAPI
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tslearn.clustering import KShape
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.clustering import silhouette_score
import seaborn as sns

In [2]:
filenames = os.listdir('exchange_data')
exchange_ids = [filename.removesuffix('.csv') for filename in filenames]

### Reading data

In [3]:
for exchange_id in exchange_ids:   
    df_ex = pd.read_csv('exchange_data/'+exchange_id+'.csv')
    df_ex['Date'] = pd.to_datetime(arg=df_ex['Date']).dt.round('H')
    mask = (df_ex['Date'] > '2023-1-1') & (df_ex['Date'] <= '2023-7-1')
    df_ex = df_ex.loc[mask]
    df_ex.to_csv('exchange_data/'+exchange_id+'.csv', index=False)

### Data integration with third party data
Missing data from 30-08-2022 to 09-09-2022 and from 16-12-2022 to 29-12-2022 has been integrated using the "volume chart" API, note that the API call provides a single datapoint for the day.  

In [4]:
def integrate_missing(exchange_id):
    #variables
    cg = CoinGeckoAPI()
    bitcoin_id = 'bitcoin'
    usd_id = 'usd'
    days_before = '32'
    n_days = 14
    
    #get current BTC price in USD
    btc_price_dict = cg.get_price(ids=bitcoin_id, vs_currencies='usd')[bitcoin_id]
    btc_price = btc_price_dict[usd_id]
    
    #prepare initial volumes
    df_ex = pd.read_csv(PATH+'./exchange_data/'+exchange_id+'.csv')
    df_ex['Date'] = pd.to_datetime(arg=df_ex['Date']).dt.round('H')
    df_ex = df_ex.dropna()
    
    try:
        #get missing data 
        res = cg.get_exchanges_volume_chart_by_id(id=exchange_id, days=days_before)
        #sleeping to avoid too many API calls
        time.sleep(0.5)
        
        df_missing = pd.DataFrame(res, columns=["Date", "Volume (USD)"], dtype=float)
        df_missing['Volume (USD)'] = df_missing['Volume (USD)'].apply(lambda x: x*btc_price)
        df_missing['Date'] = pd.to_datetime(df_missing['Date'],unit='ms')
        df_missing['Normalized Volume (USD)'] = df_missing["Volume (USD)"]
        df_missing['Date'] = df_missing['Date'].apply(lambda x: x.replace(hour=0, minute=0))
        df_missing = df_missing[:n_days]
        #integrate missing data
        df_ex = pd.concat(objs=[df_ex, df_missing], ignore_index=True)

        df_ex = df_ex.sort_values('Date')
        df_ex.to_csv(PATH+'./processed_data/'+exchange_id+'.csv', index=False)
        print("data integrated for "+exchange_id)
    except Exception as e:
        print(exchange_id+' failed..')
        print(e)

In [5]:
for exchange_id in exchange_ids:
    integrate_missing(exchange_id)

### Check missing data
Check how many datapoints (hourly volumes) are missing for every exchange and keep only exchanges that have less than 10% of missing datapoints (mainly because they didn't stay in the top 200 for long).

In [24]:
#dictionary storing the couple <exchange, n_missing>
ex_n_missing = {}
#dictionary storing the couple <exchange, missing> where "missing" are the timestamps missing
ex_missing = {}

hourly = pd.date_range(start="1-1-2023", end="7-1-2023", freq='h') #mm-dd-yyyy
hourly.freq = None
hourly = set(hourly)
print("Total datapoints (hourly)")
print(len(hourly))
def check_missing_data(exchange_id):
    df_ex = pd.read_csv('exchange_data/'+exchange_id+'.csv')
    df_ex['Date'] = pd.to_datetime(arg=df_ex['Date']).dt.round('H')
    timestamps = set(df_ex['Date'])
    missing = set(sorted(hourly)) - set(sorted(timestamps))
    missing = sorted(missing)
    #keep only exchanges that have less than 10% of missing data
    thresh = int(len(hourly)*0.10)
    if len(missing) < thresh:
        ex_n_missing[exchange_id] = len(missing)
        ex_missing[exchange_id] = missing

Total datapoints (hourly)
4345


In [25]:
for exchange_id in exchange_ids:
    check_missing_data(exchange_id)

In [26]:
sorted(ex_n_missing.items(), key=lambda x:x[1])

[('binance', 43),
 ('bingx', 43),
 ('bitbank', 43),
 ('bitflyer', 43),
 ('bitget', 43),
 ('bitmax', 43),
 ('bittrex', 43),
 ('bitvavo', 43),
 ('btcex', 43),
 ('bybit_spot', 43),
 ('coinex', 43),
 ('coin_metro', 43),
 ('crypto_com', 43),
 ('gate', 43),
 ('gdax', 43),
 ('huobi', 43),
 ('korbit', 43),
 ('kraken', 43),
 ('kucoin', 43),
 ('lbank', 43),
 ('luno', 43),
 ('mxc', 43),
 ('okex', 43),
 ('phemex', 43),
 ('poloniex', 43),
 ('upbit', 43),
 ('wootrade', 43),
 ('bitso', 44),
 ('bitstamp', 44),
 ('dextrade', 44),
 ('nominex', 44),
 ('whitebit', 44),
 ('bitkub', 45),
 ('bitmart', 45),
 ('exmo', 45),
 ('latoken', 46),
 ('pancakeswap_new', 46),
 ('bitfinex', 47),
 ('cryptology', 47),
 ('osmosis', 48),
 ('p2pb2b', 48),
 ('binance_us', 49),
 ('bit_com', 50),
 ('gmo_japan', 50),
 ('bithumb', 53),
 ('max_maicoin', 54),
 ('nice_hash', 57),
 ('gemini', 58),
 ('coinstore', 59),
 ('btcturk', 60),
 ('coinsbit', 60),
 ('okcoin', 67),
 ('xt', 72),
 ('uniswap_v2', 74),
 ('finexbox', 90),
 ('independe

In [27]:
print("Total amount of exchanges remained")
len(ex_missing)

Total amount of exchanges remained


76

### Interpolation of missing data
A linear interpolation of the missing data has been performed, now all the exchanges have the same amount of datapoints.

In [30]:
def integrate_data(exchange_id):
    df_ex = pd.read_csv('exchange_data/'+exchange_id+'.csv')
    df_ex['Date'] = pd.to_datetime(arg=df_ex['Date']).dt.round('H')
    df_tmp = pd.DataFrame({'Date': ex_missing[exchange_id]})
    df_ex = pd.concat(objs=[df_ex, df_tmp], ignore_index=True)
    df_ex = df_ex.sort_values('Date')
    if exchange_id == "coin_metro":
        print(df_ex['Volume (USD)'][df_ex.isna().any(axis=1)])
    df_ex['Volume (USD)'] = df_ex['Volume (USD)'].interpolate(method='linear', axis=0)
    df_ex.drop(columns='Normalized Volume (USD)', inplace=True)
    df_ex.reset_index(inplace=True,drop=True)
    df_ex = df_ex[1:]
    df_ex.to_csv('detection_data/'+exchange_id+'.csv', index=False)

In [ ]:
for exchange_id in ex_missing:
   integrate_data(exchange_id)

## Data Visualization

In [ ]:
#dictionary of dataframes for the exchanges -> <exchange_id, df>
df_dict = {}

def group_volumes(exchange_id):
    df_ex = pd.read_csv('detection_data/'+exchange_id+'.csv')
    df_ex['Date'] = pd.to_datetime(arg=df_ex['Date']).dt.round('H')
    print(df_ex.shape)
    df_dict[exchange_id] = df_ex['Volume (USD)']
    
for exchange_id in ex_missing:
    group_volumes(exchange_id)

In [13]:
df_binance = pd.read_csv(os.path.join('detection_data/binance.csv'))
df_huobi = pd.read_csv(os.path.join('detection_data/huobi.csv'))
df_kraken = pd.read_csv(os.path.join('detection_data/kraken.csv'))
df_bitstamp = pd.read_csv(os.path.join('detection_data/bitstamp.csv'))
df_lbank = pd.read_csv(os.path.join('detection_data/lbank.csv'))

df_unified = pd.DataFrame(df_binance["Date"])

In [14]:
df_grouped = pd.DataFrame(df_binance, columns=list(ex_missing.keys()))

for exchange_id in ex_missing:
    df_unified = pd.concat(objs=[df_unified,df_dict[exchange_id]], axis=1)
    df_unified.rename(columns={"Volume (USD)":exchange_id}, inplace=True)
    
df_unified.dropna(inplace=True)
df_tmp = df_unified.drop(columns=["Date"])
df_avgs = [(df_tmp[x].mean(),x) for x in df_tmp.columns]
df_avgs.sort(reverse=True)
df_avgs = df_avgs[:50]
all_ex = [x[1] for x in df_avgs]


In [15]:
df_unified

,Date,bigone,binance,binance_us,bingx,bitazza,bitbank,bitbns,bitcoin_com,bitfinex,...,poloniex,sushiswap_arbitrum,tidex,uniswap_v2,uniswap_v3_arbitrum,upbit,velodrome,whitebit,wootrade,xt
196,2023-01-09 05:00:00,2.952045e+08,8.879489e+09,2.244302e+08,3.401547e+08,7.085623e+06,1.086611e+07,1.887584e+07,7.001000e+08,1.277784e+08,...,6.431085e+07,2.813677e+06,6.063927e+08,5.623671e+07,3.640453e+07,9.543139e+08,8.505038e+06,6.295274e+08,4.066790e+07,4.864805e+08
197,2023-01-09 06:00:00,3.058521e+08,9.214147e+09,2.378764e+08,3.561302e+08,7.093880e+06,1.150019e+07,2.070109e+07,7.284573e+08,1.311485e+08,...,6.522371e+07,3.092162e+06,6.391415e+08,5.639027e+07,3.775739e+07,9.904522e+08,9.199753e+06,6.653282e+08,4.205730e+07,4.986783e+08
198,2023-01-09 07:00:00,3.126778e+08,9.584539e+09,2.460826e+08,3.679820e+08,7.182646e+06,1.199381e+07,2.036891e+07,7.580263e+08,1.356727e+08,...,6.652146e+07,3.370646e+06,6.572593e+08,5.385615e+07,3.874204e+07,1.007149e+09,9.388130e+06,6.801791e+08,4.304051e+07,5.093855e+08
199,2023-01-09 08:00:00,3.166636e+08,9.809723e+09,2.508864e+08,3.790016e+08,7.288493e+06,1.266824e+07,2.064150e+07,7.834128e+08,1.395465e+08,...,6.757848e+07,3.521520e+06,6.753771e+08,5.407393e+07,3.952821e+07,1.033214e+09,9.460886e+06,6.958298e+08,4.547027e+07,5.180830e+08
200,2023-01-09 09:00:00,3.223671e+08,1.007814e+10,2.594680e+08,3.915855e+08,7.449766e+06,1.394492e+07,2.337689e+07,8.135347e+08,1.452817e+08,...,6.867471e+07,3.994906e+06,6.934949e+08,5.486714e+07,4.126680e+07,1.065712e+09,9.623184e+06,7.140912e+08,4.668097e+07,5.279182e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4339,2023-06-30 20:00:00,1.976813e+08,1.317876e+10,3.122879e+07,6.757353e+08,7.699788e+06,5.974650e+07,1.285657e+07,3.049837e+08,2.155748e+08,...,4.564468e+07,4.014573e+06,7.567505e+08,5.382383e+07,5.506411e+08,3.858478e+09,9.286812e+06,1.130229e+09,8.327256e+07,8.964306e+08
4340,2023-06-30 21:00:00,2.038511e+08,1.381663e+10,3.280561e+07,7.042209e+08,7.787404e+06,5.970721e+07,1.304446e+07,3.074846e+08,2.206522e+08,...,4.608216e+07,4.117808e+06,7.794959e+08,5.299619e+07,5.866178e+08,3.830474e+09,9.462675e+06,1.155596e+09,8.664885e+07,9.403911e+08
4341,2023-06-30 22:00:00,2.053952e+08,1.396603e+10,3.280864e+07,7.104901e+08,7.890105e+06,6.055749e+07,1.303793e+07,3.104153e+08,2.223183e+08,...,4.645544e+07,4.155671e+06,7.892288e+08,5.313762e+07,5.930525e+08,3.878621e+09,9.331110e+06,1.174265e+09,8.897326e+07,9.502223e+08
4342,2023-06-30 23:00:00,2.040205e+08,1.392482e+10,3.222486e+07,7.021052e+08,7.902051e+06,6.057257e+07,1.284713e+07,3.189556e+08,2.231920e+08,...,4.670463e+07,4.150714e+06,7.879014e+08,5.274964e+07,5.916152e+08,3.893426e+09,9.063179e+06,1.174139e+09,8.655973e+07,9.474159e+08


In [16]:
df_unified.to_csv("df_unified.csv", index=False)

In [17]:
df_final = df_unified[all_ex]
df_final['Date'] = df_unified['Date'] 
df_final.dropna(inplace=True)
df_final.drop_duplicates(inplace=True)
df_final.set_index(df_final['Date'], inplace=True)
df_final

C:\Users\lucas\AppData\Local\Temp\ipykernel_17724\2176894401.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Date'] = df_unified['Date']
C:\Users\lucas\AppData\Local\Temp\ipykernel_17724\2176894401.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final.dropna(inplace=True)
C:\Users\lucas\AppData\Local\Temp\ipykernel_17724\2176894401.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

,binance,upbit,okex,gdax,mxc,lbank,p2pb2b,digifinex,gate,tidex,...,exmo,poloniex,bitvavo,bitflyer,wootrade,cryptology,bitkub,coinex,gemini,Date
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-09 05:00:00,8.879489e+09,9.543139e+08,7.935585e+08,1.204842e+09,9.925685e+08,9.613351e+08,8.125105e+08,4.654403e+08,2.543147e+08,6.063927e+08,...,4.459763e+07,6.431085e+07,4.154412e+07,1.729554e+07,4.066790e+07,2.723599e+07,4.411966e+07,2.107683e+07,1.385551e+07,2023-01-09 05:00:00
2023-01-09 06:00:00,9.214147e+09,9.904522e+08,8.376459e+08,1.278344e+09,1.013860e+09,9.754235e+08,8.639056e+08,4.883919e+08,2.616439e+08,6.391415e+08,...,4.469543e+07,6.522371e+07,4.320626e+07,1.827545e+07,4.205730e+07,2.805015e+07,4.693043e+07,2.220722e+07,1.467809e+07,2023-01-09 06:00:00
2023-01-09 07:00:00,9.584539e+09,1.007149e+09,8.674842e+08,1.330669e+09,1.024520e+09,9.775939e+08,8.769030e+08,5.073882e+08,2.680529e+08,6.572593e+08,...,4.487045e+07,6.652146e+07,4.456422e+07,1.883079e+07,4.304051e+07,2.837637e+07,5.033132e+07,2.356768e+07,1.501321e+07,2023-01-09 07:00:00
2023-01-09 08:00:00,9.809723e+09,1.033214e+09,8.952240e+08,1.355809e+09,1.032343e+09,9.784294e+08,9.042979e+08,5.160224e+08,2.744557e+08,6.753771e+08,...,4.479678e+07,6.757848e+07,4.691787e+07,1.939938e+07,4.547027e+07,2.868671e+07,5.253996e+07,2.379350e+07,1.531069e+07,2023-01-09 08:00:00
2023-01-09 09:00:00,1.007814e+10,1.065712e+09,9.202583e+08,1.401838e+09,1.041797e+09,9.992081e+08,9.250121e+08,5.301954e+08,2.984710e+08,6.934949e+08,...,4.493820e+07,6.867471e+07,4.878949e+07,2.037457e+07,4.668097e+07,2.926895e+07,5.500026e+07,2.429048e+07,1.574604e+07,2023-01-09 09:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-30 20:00:00,1.317876e+10,3.858478e+09,2.617711e+09,2.210201e+09,1.102599e+09,7.800118e+08,1.848753e+09,1.323911e+09,1.167391e+09,7.567505e+08,...,6.806009e+07,4.564468e+07,1.344764e+08,1.091564e+08,8.327256e+07,5.603196e+07,4.442529e+07,8.313563e+07,4.779751e+07,2023-06-30 20:00:00
2023-06-30 21:00:00,1.381663e+10,3.830474e+09,2.722929e+09,2.312341e+09,1.106349e+09,8.042589e+08,1.925958e+09,1.323911e+09,1.220734e+09,7.794959e+08,...,6.792550e+07,4.608216e+07,1.320676e+08,1.105920e+08,8.664885e+07,5.729284e+07,4.468329e+07,8.675717e+07,5.015306e+07,2023-06-30 21:00:00
2023-06-30 22:00:00,1.396603e+10,3.878621e+09,2.749391e+09,2.327206e+09,1.017751e+09,8.371861e+08,1.945528e+09,1.323911e+09,1.231094e+09,7.892288e+08,...,6.813260e+07,4.645544e+07,1.319129e+08,1.110602e+08,8.897326e+07,5.746769e+07,4.493065e+07,8.820508e+07,5.198464e+07,2023-06-30 22:00:00


In [ ]:
import plotly.express as px

fig = px.line(df_unified, x="Date", y=df_unified.columns,
              hover_data={"Date": "|%B %d, %Y"},
              title='hourly volume analysis')
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")
fig.show()

In [19]:
df_unified.dropna(inplace=True)
df_unified

,Date,bigone,binance,binance_us,bingx,bitazza,bitbank,bitbns,bitcoin_com,bitfinex,...,poloniex,sushiswap_arbitrum,tidex,uniswap_v2,uniswap_v3_arbitrum,upbit,velodrome,whitebit,wootrade,xt
196,2023-01-09 05:00:00,2.952045e+08,8.879489e+09,2.244302e+08,3.401547e+08,7.085623e+06,1.086611e+07,1.887584e+07,7.001000e+08,1.277784e+08,...,6.431085e+07,2.813677e+06,6.063927e+08,5.623671e+07,3.640453e+07,9.543139e+08,8.505038e+06,6.295274e+08,4.066790e+07,4.864805e+08
197,2023-01-09 06:00:00,3.058521e+08,9.214147e+09,2.378764e+08,3.561302e+08,7.093880e+06,1.150019e+07,2.070109e+07,7.284573e+08,1.311485e+08,...,6.522371e+07,3.092162e+06,6.391415e+08,5.639027e+07,3.775739e+07,9.904522e+08,9.199753e+06,6.653282e+08,4.205730e+07,4.986783e+08
198,2023-01-09 07:00:00,3.126778e+08,9.584539e+09,2.460826e+08,3.679820e+08,7.182646e+06,1.199381e+07,2.036891e+07,7.580263e+08,1.356727e+08,...,6.652146e+07,3.370646e+06,6.572593e+08,5.385615e+07,3.874204e+07,1.007149e+09,9.388130e+06,6.801791e+08,4.304051e+07,5.093855e+08
199,2023-01-09 08:00:00,3.166636e+08,9.809723e+09,2.508864e+08,3.790016e+08,7.288493e+06,1.266824e+07,2.064150e+07,7.834128e+08,1.395465e+08,...,6.757848e+07,3.521520e+06,6.753771e+08,5.407393e+07,3.952821e+07,1.033214e+09,9.460886e+06,6.958298e+08,4.547027e+07,5.180830e+08
200,2023-01-09 09:00:00,3.223671e+08,1.007814e+10,2.594680e+08,3.915855e+08,7.449766e+06,1.394492e+07,2.337689e+07,8.135347e+08,1.452817e+08,...,6.867471e+07,3.994906e+06,6.934949e+08,5.486714e+07,4.126680e+07,1.065712e+09,9.623184e+06,7.140912e+08,4.668097e+07,5.279182e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4339,2023-06-30 20:00:00,1.976813e+08,1.317876e+10,3.122879e+07,6.757353e+08,7.699788e+06,5.974650e+07,1.285657e+07,3.049837e+08,2.155748e+08,...,4.564468e+07,4.014573e+06,7.567505e+08,5.382383e+07,5.506411e+08,3.858478e+09,9.286812e+06,1.130229e+09,8.327256e+07,8.964306e+08
4340,2023-06-30 21:00:00,2.038511e+08,1.381663e+10,3.280561e+07,7.042209e+08,7.787404e+06,5.970721e+07,1.304446e+07,3.074846e+08,2.206522e+08,...,4.608216e+07,4.117808e+06,7.794959e+08,5.299619e+07,5.866178e+08,3.830474e+09,9.462675e+06,1.155596e+09,8.664885e+07,9.403911e+08
4341,2023-06-30 22:00:00,2.053952e+08,1.396603e+10,3.280864e+07,7.104901e+08,7.890105e+06,6.055749e+07,1.303793e+07,3.104153e+08,2.223183e+08,...,4.645544e+07,4.155671e+06,7.892288e+08,5.313762e+07,5.930525e+08,3.878621e+09,9.331110e+06,1.174265e+09,8.897326e+07,9.502223e+08
4342,2023-06-30 23:00:00,2.040205e+08,1.392482e+10,3.222486e+07,7.021052e+08,7.902051e+06,6.057257e+07,1.284713e+07,3.189556e+08,2.231920e+08,...,4.670463e+07,4.150714e+06,7.879014e+08,5.274964e+07,5.916152e+08,3.893426e+09,9.063179e+06,1.174139e+09,8.655973e+07,9.474159e+08
